# SQL-Based Business Analysis

## Goal
Reproduce core business analytics using SQL to demonstrate
SQL fluency and analytics engineering skills.

## Setup and Data Registration

DuckDB is used to execute SQL queries locally.
Cleaned pandas DataFrames are registered as SQL tables.

In [9]:
import duckdb
import pandas as pd

# Load exported cleaned data
df_revenue = pd.read_csv("../data/cleaned_revenue.csv", parse_dates=["InvoiceDate"])
df_customers = pd.read_csv("../data/cleaned_customers.csv", parse_dates=["InvoiceDate"])

con = duckdb.connect()

# Register as SQL tables
con.register("revenue_txn", df_revenue)
con.register("customer_txn", df_customers)

# sanity check
con.execute("SELECT COUNT(*) AS n FROM revenue_txn").df()


/var/folders/6k/5vvm5_jd7_gcx3x7fxynfbdr0000gn/T/ipykernel_76556/3186597715.py:5: DtypeWarning: Columns (0: InvoiceNo) have mixed types. Specify dtype option on import or set low_memory=False.
  df_revenue = pd.read_csv("../data/cleaned_revenue.csv", parse_dates=["InvoiceDate"])


,n
0,530104


## Monthly Revenue KPIs (SQL)

This query aggregates transaction-level data into
monthly business performance metrics.

In [10]:
# Monthly KPIs
monthly_sql = """
SELECT
  invoice_month,
  COUNT(DISTINCT InvoiceNo) AS orders,
  SUM(CASE WHEN line_revenue > 0 THEN line_revenue ELSE 0 END) AS gross_revenue,
  SUM(CASE WHEN line_revenue < 0 THEN line_revenue ELSE 0 END) AS returns,
  SUM(line_revenue) AS net_revenue,
  SUM(line_revenue) / COUNT(DISTINCT InvoiceNo) AS aov
FROM revenue_txn
GROUP BY 1
ORDER BY 1;
"""
monthly_kpis_sql = con.execute(monthly_sql).df()
monthly_kpis_sql.head()

,invoice_month,orders,gross_revenue,returns,net_revenue,aov
0,2010-12,1559,823746.140,0.0,823746.140,528.381103
1,2011-01,1086,691364.560,0.0,691364.560,636.615617
2,2011-02,1100,523631.890,0.0,523631.890,476.028991
3,2011-03,1454,717639.360,0.0,717639.360,493.562146
4,2011-04,1246,537808.621,0.0,537808.621,431.628107


## Customer-Level Summary (SQL)

Customer-level metrics are computed using SQL,
mirroring the logic implemented earlier in pandas.

In [11]:
# Customer summary (top 10 by revenue)
customer_sql = """
SELECT
  CustomerID,
  COUNT(DISTINCT InvoiceNo) AS total_orders,
  SUM(line_revenue) AS total_revenue,
  MIN(InvoiceDate) AS first_purchase,
  MAX(InvoiceDate) AS last_purchase,
  DATE_DIFF('day', MAX(InvoiceDate), (SELECT MAX(InvoiceDate) FROM customer_txn)) * -1 AS recency_days
FROM customer_txn
WHERE CustomerID IS NOT NULL
GROUP BY 1
ORDER BY total_revenue DESC
LIMIT 10;
"""
con.execute(customer_sql).df()

,CustomerID,total_orders,total_revenue,first_purchase,last_purchase,recency_days
0,14646.0,73,280206.02,2010-12-20 10:09:00,2011-12-08 12:12:00,-1
1,18102.0,60,259657.30,2010-12-07 16:42:00,2011-12-09 11:50:00,0
2,17450.0,46,194550.79,2010-12-07 09:23:00,2011-12-01 13:29:00,-8
3,16446.0,2,168472.50,2011-05-18 09:52:00,2011-12-09 09:15:00,0
4,14911.0,201,143825.06,2010-12-01 14:05:00,2011-12-08 15:54:00,-1
5,12415.0,21,124914.53,2011-01-06 11:12:00,2011-11-15 14:22:00,-24
6,14156.0,55,117379.63,2010-12-03 11:48:00,2011-11-30 10:54:00,-9
7,17511.0,31,91062.38,2010-12-01 10:19:00,2011-12-07 10:12:00,-2
8,16029.0,63,81024.84,2010-12-01 09:57:00,2011-11-01 10:27:00,-38
9,12346.0,1,77183.60,2011-01-18 10:01:00,2011-01-18 10:01:00,-325


## Cohort Retention Base Table (SQL)

This query assigns cohort month and cohort index
to each transaction in preparation for retention analysis.

In [12]:
# Cohort counts (cohort_month x cohort_index)
cohort_sql = """
WITH first_purchase AS (
  SELECT
    CustomerID,
    STRFTIME(MIN(InvoiceDate), '%Y-%m') AS cohort_month
  FROM customer_txn
  WHERE CustomerID IS NOT NULL
  GROUP BY 1
),
tx AS (
  SELECT
    t.CustomerID,
    f.cohort_month,
    STRFTIME(t.InvoiceDate, '%Y-%m') AS invoice_month
  FROM customer_txn t
  JOIN first_purchase f USING (CustomerID)
),
indexed AS (
  SELECT
    cohort_month,
    invoice_month,
    DATE_DIFF('month',
      CAST(cohort_month || '-01' AS DATE),
      CAST(invoice_month || '-01' AS DATE)
    ) AS cohort_index,
    CustomerID
  FROM tx
)
SELECT
  cohort_month,
  cohort_index,
  COUNT(DISTINCT CustomerID) AS customers
FROM indexed
GROUP BY 1, 2
ORDER BY 1, 2;
"""
cohort_counts_sql = con.execute(cohort_sql).df()
cohort_counts_sql.head(10)

,cohort_month,cohort_index,customers
0,2010-12,0,885
1,2010-12,1,324
2,2010-12,2,286
3,2010-12,3,340
4,2010-12,4,321
5,2010-12,5,352
6,2010-12,6,321
7,2010-12,7,309
8,2010-12,8,313
9,2010-12,9,350


## Interpretation

The SQL results match the pandas-based analysis,
confirming consistency across implementations.

This demonstrates the ability to translate
analytics logic between Python and SQL.

## Cohort Retention Matrix (SQL)

We compute customer retention percentages by dividing
active customers in each cohort/month by the cohort size
at month 0.

In [13]:
cohort_retention_sql = """
WITH first_purchase AS (
  SELECT
    CustomerID,
    STRFTIME(MIN(InvoiceDate), '%Y-%m') AS cohort_month
  FROM customer_txn
  WHERE CustomerID IS NOT NULL
  GROUP BY 1
),
tx AS (
  SELECT
    t.CustomerID,
    f.cohort_month,
    STRFTIME(t.InvoiceDate, '%Y-%m') AS invoice_month
  FROM customer_txn t
  JOIN first_purchase f USING (CustomerID)
),
indexed AS (
  SELECT
    cohort_month,
    DATE_DIFF(
      'month',
      CAST(cohort_month || '-01' AS DATE),
      CAST(invoice_month || '-01' AS DATE)
    ) AS cohort_index,
    CustomerID
  FROM tx
),
cohort_counts AS (
  SELECT
    cohort_month,
    cohort_index,
    COUNT(DISTINCT CustomerID) AS customers
  FROM indexed
  GROUP BY 1, 2
),
cohort_sizes AS (
  SELECT
    cohort_month,
    customers AS cohort_size
  FROM cohort_counts
  WHERE cohort_index = 0
)
SELECT
  c.cohort_month,
  c.cohort_index,
  ROUND(c.customers * 1.0 / s.cohort_size, 4) AS retention_rate
FROM cohort_counts c
JOIN cohort_sizes s USING (cohort_month)
ORDER BY 1, 2;
"""
cohort_retention_sql_df = con.execute(cohort_retention_sql).df()
cohort_retention_sql_df.head(10)

,cohort_month,cohort_index,retention_rate
0,2010-12,0,1.0000
1,2010-12,1,0.3661
2,2010-12,2,0.3232
3,2010-12,3,0.3842
4,2010-12,4,0.3627
5,2010-12,5,0.3977
6,2010-12,6,0.3627
7,2010-12,7,0.3492
8,2010-12,8,0.3537
9,2010-12,9,0.3955


In [14]:
monthly_kpis_sql.to_csv("../data/sql_monthly_kpis.csv", index=False)
cohort_retention_sql_df.to_csv("../data/sql_cohort_retention.csv", index=False)

print("Saved SQL outputs to data/")

Saved SQL outputs to data/


## SQL Cohort Retention Validation

The SQL-based cohort retention results match the
pandas-based analysis and visualizations exactly.

This confirms the correctness of the SQL logic and
demonstrates the ability to translate analytics
workflows between Python and SQL.